<a href="https://colab.research.google.com/github/AfsanehHabibi/reddit-conversation-quality/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install anytree

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 1.5 MB/s eta 0:00:00


In [ ]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=154d060cef5f68001c2ec3c58bf8a463bc0b626895485657b697e35f937f9556
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
!pip install  kili

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.9/209.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 10.3 MB/s eta 0:00:00
  Created wheel for cuid: filename=cuid-0.4-py2.py3-none-any.whl size=4714 sha256=f2b3c543a2cc5ebdb78caf73b7dc3fd31537448cea5c6ff4d3814e675ca569a6
  Stored in directory: /root/.cache/pip/wheels/52/0a/dc/a0e28c435d5a74d9eef3d7c3cd147b96cb21e71e5ec7dcfdbe
Successfully built cuid
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Preprocessing

In [ ]:
import numpy as np
import json
from anytree import Node, RenderTree

In [ ]:
base_path = "/content/drive/MyDrive/University/RedditData/"

In [ ]:
def extract_files_list(file_list_path):
    files_name = []
    with open(file_list_path, 'r') as file_list:
        for line in file_list:
            line = line.strip()
            if line:
                file_path, _ = line.split(',')
                files_name.append(file_path)
    return files_name

In [ ]:
data_files_name = extract_files_list(f"{base_path}timestamps_seed2_num20_period10_date2022-10-1.txt_comments_progress.txt")

In [ ]:
def make_submission_file_path(file_name):
    return f"{base_path}submissions_{file_name}.json"

In [ ]:
def make_comment_file_path(file_name):
    return f"{base_path}comments_{file_name}.json"

In [ ]:
subreddits = set()

In [ ]:
def extract_submissions_text_with_filter(data_files_name):
    submissions_dic = {}
    over_18_count = 0
    for file_name in data_files_name:
        file_path = make_submission_file_path(file_name)
        with open(file_path, 'r') as file:
            lines_sub = file.readlines()
        for l in lines_sub:
            obj = json.loads(l)
            if obj['over_18'] == False:
                submissions_dic[obj['id']]  = obj['title'] + obj['selftext']
                subreddits.add(obj['subreddit'])
            else:
                over_18_count += 1
    print("over 18 count ", over_18_count)
    return submissions_dic

In [ ]:
subsmissions = extract_submissions_text_with_filter(data_files_name)

over 18 count  66876


In [ ]:
len(subsmissions)

96333

In [ ]:
import json

def count_comments_in_file(file_path):
    count = 0
    with open(file_path, 'r') as json_file:
        lines = json_file.readlines()
        for line in lines:
            data = json.loads(line)
            count += len(data.get('comments', []))
    return count

def total_comments_in_files(file_list_path):
    total_comments = 0
    with open(file_list_path, 'r') as file_list:
        for line in file_list:
            print(line)
            line = line.strip()
            if line:
                file_path, _ = line.split(',')
                total_comments += count_comments_in_file(f"{base_path}comments_{file_path}.json")
    return total_comments

In [ ]:
def load_comments_with_filter(files_name, submmissions):
    # Create a dictionary to store the comment nodes by their IDs
    conversations_dic = {}
    for file_name in data_files_name:
        comment_file = make_comment_file_path(file_name)
        with open(comment_file, 'r') as file:
            lines = file.readlines()

        for line in lines:
            comment_nodes = {}
            json_obj = json.loads(line)
            submission_id = json_obj['submission_id']
            comments = json_obj['comments']
            if len(comments) == 0:
              continue
            if submission_id not in subsmissions:
                #submission has removed in preprocessing
                continue
            flat_comments_list = []
            parent_child_pairs = []
            # Create root node for the submission
            submission_text = subsmissions[submission_id]
            if submission_text.endswith('[removed]'):
              continue
            root = Node(submission_id, body = submission_text)
            comment_nodes[submission_id] = root
            conversation_text = submission_text
            flat_comments_list.append({'id': submission_id, 'body': submission_text, 'author':None, 'reply_to':None, 'conversation_id':submission_id})
            # Create child nodes for the comments
            for comment in comments:
                # Check if comment is from a bot and ignore it if it is
                if "I am a bot, and this action was performed automatically" in comment['body']:
                    continue

                comment_id = comment['id']
                author = comment.get('author_fullname', 'unknown')
                parent_id = comment['parent_id'].split('_', 1)[-1]
                body = comment['body']
                comment_node = Node(comment_id, body=body)

                if parent_id in comment_nodes:
                    flat_comments_list.append({'id': comment_id, 'body': body, 'author':author, 'reply_to':parent_id, 'conversation_id':submission_id})
                    parent_node = comment_nodes[parent_id]
                    parent_child_pairs.append({
                        'comment': parent_node.body,
                        'reply': body,
                        'comment_id': parent_node.name,
                        'reply_id': comment_id,
                        'global_id': parent_node.name + "_" + comment_id
                    })
                    comment_node.parent = parent_node
                    conversation_text += "/n"+ body
                    comment_nodes[comment_id] = comment_node
            # Print the tree structure
            # print(RenderTree(root))
            conversations_dic[submission_id] = {
                'tree': root,
                'full_text': conversation_text,
                'pairs': parent_child_pairs,
                'comments': flat_comments_list
            }
            #for parent in root.children:
            #  for child in parent.children:
            #       parent_child_pairs.append((parent.body, child.body))

            # Print the parent-child pairs
            #for pair in parent_child_pairs:
            #   print(f"Parent: {pair[0]}, Child: {pair[1]}")

            # Print the tree structure
            #for pre, fill, node in RenderTree(root):
            #   print(f"{pre}{node.name}")

        #print(RenderTree(conversations_tree[2555]))
    return conversations_dic

In [ ]:
conversations_dic = load_comments_with_filter(data_files_name, subsmissions)

In [ ]:
len(conversations_dic)

37340

In [ ]:
def filter_empty_conversations(conversations_dic):
  non_empty_conversations = {}
  for id in conversations_dic:
    new_pairs = []
    new_comments = []
    pairs = conversations_dic[id]['pairs']
    for pair in pairs:
      #use endswith instead of eqality check cause if a submission is removed its title might be still there
      if not(pair['comment'] == '[deleted]' or pair['reply'] == '[deleted]' or pair['comment'] == '[removed]' or pair['reply'] == '[removed]'):
        new_pairs.append(pair)
    # should i remove comments which are deleted or removed? problem is later on when measuring politness it need a whole graph and if i remove these it ends up with ids pointing to nothing
    if len(new_pairs) > 0:
      conversations_dic[id]['pairs'] = new_pairs
      non_empty_conversations[id] = conversations_dic[id]
  return non_empty_conversations


In [ ]:
non_empty_conversations = filter_empty_conversations(conversations_dic)

In [ ]:
len(non_empty_conversations)

34782

In [ ]:
from langdetect import detect, LangDetectException

def filter_non_english(conversations_dic):
    english_conversations = {}
    for id in conversations_dic:
        try:
            language = detect(conversations_dic[id]['full_text'])
            if language == 'en':
                english_conversations[id] = conversations_dic[id]
        except LangDetectException as e:
            # Handle the exception and print the problematic input
            # print(f"Language detection error for comment: {conversations_dic[id]['full_text']}")
            continue
    return english_conversations

In [ ]:
english_conversations = filter_non_english(non_empty_conversations)

In [ ]:
len(english_conversations)

32990

In [ ]:
import pickle

# Writing to a file using Pickle
with open(f"{base_path}conversations.pkl", 'wb') as file:
    pickle.dump(english_conversations, file)

print("Variables written to the file.")

# Reading from a file using Pickle
with open(f"{base_path}conversations.pkl", 'rb') as file:
    loaded_data = pickle.load(file)

# Print the loaded variables
print("Len loaded data:", len(loaded_data))


Variables written to the file.
Len loaded data: 32990


In [ ]:
import re

def contains_question(text):
    # Check if the text contains a question mark
    if '?' in text:
        return True

    # Check if the text contains certain question words
    question_words = ['what', 'how', 'when', 'where', 'who', 'why']
    for word in question_words:
        if re.search(r'\b{}\b'.format(word), text, re.IGNORECASE):
            return True

    # Check if the text contains the word "please"
    if re.search(r'\bplease\b', text, re.IGNORECASE):
        return True

    # If none of the above conditions are met, return False
    return False

# Example usage
text = "What is the capital of France? I love Paris. How do you make a cake? The recipe is easy to follow. Can you please pass the salt?"
if contains_question(text):
    print("The text contains a question.")
else:
    print("The text does not contain a question.")

The text contains a question.


In [ ]:
import random

random_pair = random.choice(list(english_conversations.items()))
for pair in random_pair[1]['pairs']:
  print(contains_question(pair['comment']))
  print("comment:", pair['comment'])
  print("reply:", pair['reply'])


True
comment: Meet up before the show?Hey, guys! Very much looking forward to these Red Rocks shows! I know it's been asked a lot on here already but would anyone be interested in meeting up for a drink on Saturday or Monday before the show? 

I work a ton and won't be able to make it to the meetup on Sunday...
reply: Yes! I’ll PM you!!


In [ ]:
import random


# Define the sample size (e.g., 3 for a sample of 3 elements)
sample_size = 100
random_pairs = []

# Use random.sample to get a random sample of the specified size
random_sample = random.sample(english_conversations.items(), sample_size)
for sample_conversation in random_sample:
    random_pair = random.choice(sample_conversation[1]['pairs'])
    random_pairs.append(random_pair)

<ipython-input-33-0df81c083ef2>:9: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random_sample = random.sample(english_conversations.items(), sample_size)


In [ ]:
len(random_pairs)

100

In [ ]:
random_pairs[:5]

[{'comment': 'Does not make it okay, does it?',
  'reply': '“it’s not okay.” is about the worst argument that anyone can come up with in relation to literally any problem.',
  'comment_id': 'iqz6fvw',
  'reply_id': 'iqz94rp',
  'global_id': 'iqz6fvw_iqz94rp'},
 {'comment': 'TrueNAS + Plex: How do get Plex to see my files?Apologies if this is not the correct subreddit.\n\nI have Plex set up through my TrueNAS server. I have two pools - GLaDOS and tank. GLaDOS holds the media files (movies ,tv shows, music), and tank holds the jail for Plex.\n\nI have mount points set up to create a link between tank and GLaDOS so that way Plex sees the folders for the libraries. I also had to give full access to @everyone in order for this to happen.\n\nNow, Plex sees the media folders- and it sees the files inside of the folders, but when I perform a library scan - it does not populate the Plex files in the library.\nHere is an [imgur album](https://imgur.com/a/NurYIOc) of what I am describing. \n\nHow

In [ ]:
import os

from kili.client import Kili
kili = Kili(api_key='6acd489f-9746-4ac4-9650-54e2d9c9faba')
interface = {
    "jobs": {
        "JOB_0": {
            "mlTask": "CLASSIFICATION",
            "required": 1,
            "isChild": False,
            "content": {
                "categories": {"IsConversation": {"name": "has a common purpose or set of purposes, or at least a mutually accepted direction"}, "NotConversation": {"name": "has no prupose or a direction"}},
                "input": "radio",
            },
        },
        "JOB_1": {
            "mlTask": "CLASSIFICATION",
            "required": 1,
            "isChild": False,
            "content": {
                "categories": {"Related": {"name": "is relevent"}, "UnRelated": {"name": "is not relevent"}},
                "input": "radio",
            },
        },
        "JOB_2": {
            "mlTask": "CLASSIFICATION",
            "required": 1,
            "isChild": False,
            "content": {
                "categories": {"HasQuality": {"name": "has quality: does not say which it lack adequate evidence or it believes to be false."}, "NonQuality": {"name": "no quality: says which it lacks evidence or it believes is false."}},
                "input": "radio",
            },
        },
        "JOB_3": {
            "mlTask": "CLASSIFICATION",
            "required": 1,
            "isChild": False,
            "content": {
                "categories": {"HasQuantity": {"name": "has quantity: is as informative as is required"}, "NoQuantity": {"name": "no  quantity: is more or less informative than is required"}},
                "input": "radio",
            },
        },
        "JOB_4": {
            "mlTask": "CLASSIFICATION",
            "required": 1,
            "isChild": False,
            "content": {
                "categories": {"HasManner": {"name": "has manner: avoids vagueness, avoids ambiguty, do not play with words, is orderly"}, "NoManner": {"name": "no manner: is vague or ambiguous or plays with words or is not orderly"}},
                "input": "radio",
            },
        },
        "JOB_5": {
            "mlTask": "CLASSIFICATION",
            "required": 1,
            "isChild": False,
            "content": {
                "categories": {"IsPolite": {"name": "is polite"}, "NotPolite": {"name": "is not polite"}},
                "input": "radio",
            },
        }
    }
}

project = kili.create_project(
    title="Conversation and Maxims",
    description="Project Description",
    input_type="TEXT",
    json_interface=interface,
)
project_id = project['id']
print("Project ID: ", project_id)
assets = kili.append_many_to_dataset(
    project_id=project_id,
    content_array = [('comment: ' + d['comment'] + '\n' + 'reply: ' + d['reply']) for d in random_pairs[:100]],
    external_id_array = [d['global_id'] for d in random_pairs[:100]]
)
# Project ID:  clp5pxaet1wdn086r7y3x0ngo

Project ID:  clp5pxaet1wdn086r7y3x0ngo


In [ ]:
import random


# Define the sample size (e.g., 3 for a sample of 3 elements)
sample_size = 1

# Use random.sample to get a random sample of the specified size
random_sample = random.sample(conversations_tree, sample_size)
from anytree import PreOrderIter

# Assuming 'root' is the root node of the conversation tree
for node in PreOrderIter(random_sample[0]):
    if node.parent is not None:
        print(f"{node.name}: {node.body}")

random_sample

iqmyeuo: Hey, we launched a new music-scanning feature at Soundslice just a few days ago ([see here](https://www.soundslice.com/blog/226/pdf-and-photo-scanning-beta/)). It uses artificial intelligence to convert PDFs/images into editable sheet music.

I'm happy to run a PDF through it for you if you'd like — just send me a DM!


[Node('/xsyewi', body="Has Anyone Tried Scanscore ?I've been trying to find a pdf scanner app or program that can scan sheet music and turn it into a **musicxml** format. I came across Scanscore, but I don't if it's any good. anyone in this sub try it? would you recommend it?")]

In [ ]:
!pip install  kili

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.7/288.7 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.6/72.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.9/209.9 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.0 MB/s eta 0:00:00
  Created wheel for cuid: filename=cuid-0.4-py2.py3-none-any.whl size=4712 sha256=b59c7136e6a0c35c44357a37f14df5f03d3a688b3754ee601d531698bd5923bd
  Stored in directory: /root/.cache/pip/wheels/52/0a/dc/a0e28c435d5a74d9eef3d7c3cd147b96cb21e71e5ec7dcfdbe
Successfully built cuid
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the

In [ ]:
from anytree import Node, RenderTree

# Create a list to store parent-child pairs
parent_child_pairs = []

# Access child nodes from the parent node and add the pairs to the list
for parent in root.children:
    for child in parent.children:
        parent_child_pairs.append((parent.name, child.name))

# Print the parent-child pairs
for pair in parent_child_pairs:
    print(f"Parent: {pair[0]}, Child: {pair[1]}")

# Print the tree structure
for pre, fill, node in RenderTree(root):
    print(f"{pre}{node.name}")


Parent: iqmum2s, Child: iqmunnz
Parent: iqmum2s, Child: iqmuns8
Parent: iqon7tc, Child: iqrmqmy
xsy76f
├── iqmum2s
│   ├── iqmunnz
│   └── iqmuns8
├── iqob6q0
└── iqon7tc
    └── iqrmqmy
        └── iqsvfoq


In [ ]:
import json

for comment in flat_comments_list:


Data has been prepared for Kili labeling.


In [ ]:
formatted_data = []
for comment in flat_comments_list:
    formatted_comment = {
        "row_data": comment['body'],
        "global_key": comment['id'],
        "media_type": "TEXT",
        "metadata_fields": [],
        "attachments": []
    }
    formatted_data.append(formatted_comment)

In [ ]:
with open(base_path +"flat_comments_list.json", "w") as json_file:
    json.dump(flat_comments_list, json_file, indent=2)

print("Data has been written to formatted_comments.json")


Data has been written to formatted_comments.json
